In [0]:
%pip install --upgrade --force-reinstall nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━ 20.5/40.5 kB 653.9 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━ 30.7/40.5 kB 654.2 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 345.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━ 51.2/57.7 kB 1.8 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.8 MB/s eta 0:00:01
   ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/1.5 MB 2.6 MB/s eta 0:00:01
   ━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/1.5 MB 1.5 MB/s eta 0:00:01
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.3 MB/s eta 0:00:01
   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/1.5 MB 2.4 MB/s eta 0:00:01
   ━

In [0]:
dbutils.library.restartPython()


In [0]:
import nltk
nltk.download('vader_lexicon')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-0571c865-d72b-4ad6-bba4-6c/nltk_data...


True

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
print(sia.polarity_scores("test for lab4!"))


{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [0]:
# Imports and Setup (All Libraries)
import re
import emoji
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand

# Create / get Spark session
spark = SparkSession.builder.getOrCreate()

# Set your storage account key
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104384.dfs.core.windows.net",
    "6GsPhkGZNxjNX3ph9pR77Kb5jZVywf/ZnBgwaKQPBtToe+sBA9pAoHqA5w7Ls5atRqGPbG8CyhNu+ASts8Yhzw=="
)
print("Key set: OK")


Key set: OK


In [0]:
# Delete old feature_v2 and features_v2 (safe - only these, Lab 3 safe)
dbutils.fs.rm("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/feature_v2/", recurse=True)
dbutils.fs.rm("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/features_v2/", recurse=True)
print("Old folders deleted: OK - Ready to recreate")

Old folders deleted: OK - Ready to recreate


In [0]:
# Load curated gold dataset from Lab 3
gold_df = spark.read.format("delta").load("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/curated_reviews")
gold_df = gold_df.dropDuplicates().na.drop(subset=["review_id", "book_id", "rating", "review_text"])
print("Curated rows:", gold_df.count())
gold_df.show(5)

Curated rows: 15739524
+--------------------+--------+--------------------+---------+-------------------+--------------------+------+--------------------+--------+-------+----------+
|           review_id| book_id|               title|author_id|        author_name|             user_id|rating|         review_text|language|n_votes|date_added|
+--------------------+--------+--------------------+---------+-------------------+--------------------+------+--------------------+--------+-------+----------+
|f12188942ad83e241...|  183092|Night Light (Rest...|    19811|   Terri Blackstock|83525535d07109ca7...|     4|This a great by a...|        |      0|2012-05-03|
|481daf04db0c41552...| 2662169|      Tender Morsels|   277536|      Margo Lanagan|f354d817c9462e8e9...|     4|Very rarely does ...|     eng|      0|2012-10-02|
|3b30444ddeb526191...|21619062|          Sungoddess|  5339739|        Sylvia Volk|833da7dcdbd3d2adf...|     2|I really wanted t...|        |      0|2017-04-19|
|90c38f12f6446485

In [0]:
# II. Splitting the dataset (70/15/15, stratified by rating)
fractions = {0: 0.7, 1: 0.7, 2: 0.7, 3: 0.7, 4: 0.7, 5: 0.7}
train_df = gold_df.sampleBy("rating", fractions=fractions, seed=42)
remaining_df = gold_df.exceptAll(train_df)
val_df = remaining_df.sample(fraction=0.5, seed=42)
test_df = remaining_df.exceptAll(val_df)

train_df.write.mode("overwrite").format("delta").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/feature_v2/train/")
val_df.write.mode("overwrite").format("delta").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/feature_v2/validation/")
test_df.write.mode("overwrite").format("delta").save("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/feature_v2/test/")
print("Splits saved to feature_v2/: Train", train_df.count(), "Val", val_df.count(), "Test", test_df.count())

Splits saved to feature_v2/: Train 11017216 Val 2360634 Test 3948894


In [0]:
# III.2. Load Train Split (Subsample 0.1% for memory)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")

train_df = spark.read.format("delta").load("abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/feature_v2/train/")
subsampled_train_df = train_df.sample(fraction=0.001, seed=42)
pdf = subsampled_train_df.select("review_id", "book_id", "rating", "review_text").toPandas()
print("Subsample shape:", pdf.shape)
print(pdf.head())

Subsample shape: (11128, 4)
                          review_id  ...                                        review_text
0  c4008bcffb9e7655511dfc81c0ab47e0  ...  Good story, though I didn't get any insight fr...
1  46929857411b2f4b96b8a753a2dcccd7  ...  Interesting account of cartoonist Forney's exp...
2  095b6f24c5456e0b913024847dce5cdf  ...  Once again, John Green manages to reach inside...
3  619490c1ec60b1fda24480798fc185e1  ...  My daughter, Rachael, asked me to read chapter...
4  6788802fff6cd57ccdafa729d1340073  ...  In Death of the Soul, Barrett takes a brave ex...

[5 rows x 4 columns]


In [0]:
# III.3. Text Cleaning
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '[URL]', text)
    text = re.sub(r'\d+', '[NUM]', text)
    text = emoji.replace_emoji(text, '[EMOJI]')
    text = re.sub(r'[^\w\s]', '', text)
    text = ' '.join(text.split())
    return text.strip()
pdf['cleaned_text'] = pdf['review_text'].apply(clean_text)
pdf = pdf[pdf['cleaned_text'].str.len() >= 10]
print("After cleaning shape:", pdf.shape)
print(pdf.head())

After cleaning shape: (10869, 5)
                          review_id  ...                                       cleaned_text
0  c4008bcffb9e7655511dfc81c0ab47e0  ...  good story though i didnt get any insight from it
1  46929857411b2f4b96b8a753a2dcccd7  ...  interesting account of cartoonist forneys expe...
2  095b6f24c5456e0b913024847dce5cdf  ...  once again john green manages to reach inside ...
3  619490c1ec60b1fda24480798fc185e1  ...  my daughter rachael asked me to read chapter e...
4  6788802fff6cd57ccdafa729d1340073  ...  in death of the soul barrett takes a brave exc...

[5 rows x 5 columns]


In [0]:
# III.4.a. Basic Text Features
pdf['review_length_words'] = pdf['cleaned_text'].apply(lambda x: len(x.split()))
pdf['review_length_chars'] = pdf['cleaned_text'].apply(len)
print(pdf[['cleaned_text', 'review_length_words', 'review_length_chars']].head())
print("Basic features added")

                                        cleaned_text  ...  review_length_chars
0  good story though i didnt get any insight from it  ...                   49
1  interesting account of cartoonist forneys expe...  ...                 1343
2  once again john green manages to reach inside ...  ...                  397
3  my daughter rachael asked me to read chapter e...  ...                  357
4  in death of the soul barrett takes a brave exc...  ...                 3934

[5 rows x 3 columns]
Basic features added


In [0]:
# III.4.b. Sentiment Features
sia = SentimentIntensityAnalyzer()
sentiment_scores = pdf['cleaned_text'].apply(sia.polarity_scores)
pdf['sentiment_pos'] = sentiment_scores.apply(lambda x: x['pos'])
pdf['sentiment_neg'] = sentiment_scores.apply(lambda x: x['neg'])
pdf['sentiment_neu'] = sentiment_scores.apply(lambda x: x['neu'])
pdf['sentiment_compound'] = sentiment_scores.apply(lambda x: x['compound'])
print(pdf[['cleaned_text', 'sentiment_compound']].head())
print("Sentiment features added")

                                        cleaned_text  sentiment_compound
0  good story though i didnt get any insight from it              0.4404
1  interesting account of cartoonist forneys expe...              0.9752
2  once again john green manages to reach inside ...              0.9517
3  my daughter rachael asked me to read chapter e...              0.9413
4  in death of the soul barrett takes a brave exc...              0.9959
Sentiment features added


In [0]:
# III.4.c. TF-IDF Features (Sanitized + Prefix)
tfidf = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
tfidf_matrix = tfidf.fit_transform(pdf['cleaned_text'])
invalid_chars = ' ,;{}()\\n\\t='
feature_names = ['tfidf_' + ''.join(['_' if c in invalid_chars else c for c in name]) for name in tfidf.get_feature_names_out()]
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names, index=pdf.index)
pdf = pd.concat([pdf, tfidf_df], axis=1)
print("TF-IDF shape:", tfidf_df.shape)
print("Sample names:", feature_names[:10])

TF-IDF shape: (10869, 5000)
Sample names: ['tfidf_aa_', 'tfidf_aba_do_ed', 'tfidf_abbie', 'tfidf_abby', 'tfidf_aber', 'tfidf_abigail', 'tfidf_abili_ies', 'tfidf_abili_y', 'tfidf_able', 'tfidf_abrup_']


In [0]:
# III.4.d. Semantic Embedding Features
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(pdf['cleaned_text'].tolist(), show_progress_bar=True)
embed_df = pd.DataFrame(embeddings, columns=[f'bert_emb_{i}' for i in range(embeddings.shape[1])], index=pdf.index)
pdf = pd.concat([pdf, embed_df], axis=1)
print("Embeddings shape:", embed_df.shape)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/340 [00:00<?, ?it/s]

Embeddings shape: (10869, 384)


In [0]:
# III.4.e. Additional Features (Bonus)
pdf['avg_word_length'] = pdf['cleaned_text'].apply(lambda x: sum(len(w) for w in x.split()) / len(x.split()) if x.split() else 0)
print(pdf[['cleaned_text', 'avg_word_length']].head())
print("Bonus feature added")

                                        cleaned_text  avg_word_length
0  good story though i didnt get any insight from it         4.000000
1  interesting account of cartoonist forneys expe...         4.946903
2  once again john green manages to reach inside ...         4.527778
3  my daughter rachael asked me to read chapter e...         4.188406
4  in death of the soul barrett takes a brave exc...         5.206625
Bonus feature added


In [0]:
#  Inspect current pdf
print("pdf shape:", pdf.shape)

# Cap training size to something safe for TF-IDF + write
MAX_ROWS = 20000  # you can lower to 10000 if your cluster is small

if len(pdf) > MAX_ROWS:
    pdf_small = pdf.sample(n=MAX_ROWS, random_state=42)
    print(f"Using a sampled subset of pdf: {pdf_small.shape}")
else:
    pdf_small = pdf.copy()
    print("Using full pdf (already small enough):", pdf_small.shape)

# We'll use pdf_small from now on instead of pdf


pdf shape: (10869, 5396)
Using full pdf (already small enough): (10869, 5396)


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# IV. Combined Feature Set and Output (SAFE VERSION)

# 1) Drop raw text columns & duplicates on the small subset
final_pdf = pdf_small.drop(columns=['review_text', 'cleaned_text'], errors='ignore')
final_pdf = final_pdf.loc[:, ~final_pdf.columns.duplicated()].copy()

print("final_pdf shape (after dropping text & dups):", final_pdf.shape)

# 2) Convert to Spark DataFrame
final_df = spark.createDataFrame(final_pdf)

# 3) Optionally repartition to avoid skew when writing
final_df = final_df.repartition(100)

# 4) Make sure the target path is clean to avoid schema conflicts
target_path = "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/features_v2/train/"

try:
    dbutils.fs.rm(target_path, recurse=True)
    print(f"Removed old Delta table at: {target_path}")
except Exception as e:
    print(f"Could not remove existing path (maybe it did not exist): {e}")

# 5) Write as Delta (no mergeSchema needed now)
final_df.write.mode("overwrite").format("delta") \
    .option("delta.columnMapping.mode", "name") \
    .option("delta.minReaderVersion", "2") \
    .option("delta.minWriterVersion", "5") \
    .save(target_path)

print("✅ Featured train saved to features_v2/train/")

# 6) Sanity check: read back and inspect
train_check = spark.read.format("delta").load(target_path)
print("Rows in saved train features:", train_check.count())
print("Number of columns:", len(train_check.columns))
train_check.printSchema()
train_check.show(5, truncate=False)


final_pdf shape (after dropping text & dups): (10869, 5360)
Removed old Delta table at: abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/features_v2/train/
✅ Featured train saved to features_v2/train/
Rows in saved train features: 10869
Number of columns: 5360
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- rating: long (nullable = true)
 |-- review_length_words: long (nullable = true)
 |-- review_length_chars: long (nullable = true)
 |-- sentiment_pos: double (nullable = true)
 |-- sentiment_neg: double (nullable = true)
 |-- sentiment_neu: double (nullable = true)
 |-- sentiment_compound: double (nullable = true)
 |-- tfidf_aa_: double (nullable = true)
 |-- tfidf_aba_do_ed: double (nullable = true)
 |-- tfidf_abbie: double (nullable = true)
 |-- tfidf_abby: double (nullable = true)
 |-- tfidf_aber: double (nullable = true)
 |-- tfidf_abigail: double (nullable = true)
 |-- tfidf_abili_ies: double (nullable = true)
 |-- tfi

In [0]:
# Verification: Run After IV to Confirm Lab Done
try:
    path = "abfss://lakehouse@goodreadsreviews60104384.dfs.core.windows.net/gold/features_v2/train/"
    features_df = spark.read.format("delta").load(path)
    print("Loaded: OK")
    
    schema = features_df.schema
    columns = [field.name for field in schema]
    types = {field.name: field.dataType.simpleString() for field in schema}
    
    expected_metadata = {'review_id', 'book_id', 'rating'}
    expected_basic = {'review_length_words', 'review_length_chars'}
    expected_sentiment = {'sentiment_pos', 'sentiment_neg', 'sentiment_neu', 'sentiment_compound'}
    expected_bonus = {'avg_word_length'}
    
    # ----- Column presence checks -----
    if expected_metadata.issubset(columns):
        print("Metadata: OK")
    else:
        print("Missing metadata: ERROR", expected_metadata - set(columns))
    
    if expected_basic.issubset(columns):
        print("Basic: OK")
    else:
        print("Missing basic: ERROR", expected_basic - set(columns))
    
    if expected_sentiment.issubset(columns):
        print("Sentiment: OK")
    else:
        print("Missing sentiment: ERROR", expected_sentiment - set(columns))
    
    tfidf_cols = [c for c in columns if c.startswith('tfidf_')]
    if len(tfidf_cols) >= 100:
        print("TF-IDF (~5000): OK (found", len(tfidf_cols), ")")
    else:
        print("TF-IDF incomplete: ERROR (found", len(tfidf_cols), ")")
    
    embed_cols = [c for c in columns if c.startswith('bert_emb_')]
    if 300 <= len(embed_cols) <= 400:
        print("Embeddings (~384): OK (found", len(embed_cols), ")")
    else:
        print("Embeddings missing: ERROR (found", len(embed_cols), ")")
    
    if 'avg_word_length' in columns:
        print("Bonus: OK")
    else:
        print("Bonus missing: WARNING (optional)")
    
    # ----- Type checks (more flexible for integer cols) -----
    type_errors = []

    # rating, review_length_*: accept both int and bigint
    int_like_ok = {'int', 'bigint'}
    if 'rating' in types and types['rating'] not in int_like_ok:
        type_errors.append('rating')
    if 'review_length_words' in types and types['review_length_words'] not in int_like_ok:
        type_errors.append('review_length_words')
    if 'review_length_chars' in types and types['review_length_chars'] not in int_like_ok:
        type_errors.append('review_length_chars')
    
    # sentiment + bonus: must be double if present
    double_expected = {
        'sentiment_pos',
        'sentiment_neg',
        'sentiment_neu',
        'sentiment_compound',
    }
    if 'avg_word_length' in columns:
        double_expected.add('avg_word_length')
    
    for c in double_expected:
        if c in types and types[c] != 'double':
            type_errors.append(c)
    
    if not type_errors:
        print("Types: OK")
    else:
        print("Type errors: ERROR in", type_errors)
    
    # ----- Row count -----
    row_count = features_df.count()
    if row_count > 0:
        print("Rows: OK (", row_count, ")")
    else:
        print("No rows: ERROR")
    
    print("\nSample:")
    features_df.show(5, truncate=50)
    
    print("\nIf all 'OK' and sample good, Lab 4 complete! Export to GitHub and submit.")

except Exception as e:
    print("Failed: ERROR -", str(e))
    print("Re-run IV if needed.")


Loaded: OK
Metadata: OK
Basic: OK
Sentiment: OK
TF-IDF (~5000): OK (found 4966 )
Embeddings (~384): OK (found 384 )
Bonus: OK
Types: OK
Rows: OK ( 10869 )

Sample:
+--------------------------------+--------+------+-------------------+-------------------+-------------+-------------+-------------+------------------+---------+---------------+-----------+----------+----------+-------------+---------------+-------------+----------+------------+--------------+----------------+---------------------+----------------------+-----------+------------+-------------+--------------+-------------+------------+------------+----------------+--------------+---------------+------------+----------------+--------------+------------------+------------------+---------------+-------------+--------------+--------------+-------------+-------------+---------+-----------+------------+------------+-------------------+-------------+----------+------------+--------------+-------------------+--------+---------+-------